## Conversion of tabular data to other formats for querying 
The purpose of this notebook is to test (memory, time) efficient methods to work with big data that don't fit into memory for queries and to run downstream analyses. In this case the data are large tabular files of mobile data. 
The two methods I will try are: 

1) To create a database [with SQLite](https://www.sqlite.org/index.html) (in this notebook)
2) To re-write the data to a parquet format using [Apache Arrow](https://arrow.apache.org/docs/python/parquet.html) (in the `tabular_to_parquet.ipynb` notebook)

### Data reading and package imports shared across methods: 

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
import glob
#from tqdm import tqdm, notebook
from tqdm.notebook import trange, tqdm

import pandas as pd
import dask.dataframe as dd
import geopandas as gpd
from datetime import datetime as dt

import mobilkit #.loader.crop_spatial as mkcrop_spatial

# Access environment variables and define other necessary variables
data_dir = os.getenv('WORKING_DIR')
meta_dir = f'{data_dir}metadata/'

data_2019 = f'{data_dir}data/year=2019/'
data_folders = glob.glob((data_2019 + '*/'))
print(data_folders)

initial_cols=['device_id', 'id_type', 'latitude', 'longitude', 'horizontal_accuracy', 'timestamp',  'ip_address', 'device_os', 'country', 'unknown_2', 'geohash']
sel_cols = ["device_id","latitude","longitude","timestamp","geohash","horizontal_accuracy"]
final_cols = ["uid","lat","lng","datetime","geohash","horizontal_accuracy"]

# boundary box that roughly captures the larger county of Bogota
minlon = -74.453
maxlon = -73.992
minlat = 3.727
maxlat = 4.835


In [ ]:
#### FUNCTIONS FOR DATA PROCESSING ####

def get_days(data_folder):
    """Assuming a directory organized as a month's worth of days with files in each directory like "day=01", etc """
    day_dirs = glob.glob((data_folder + '*/'))
    return day_dirs

def get_files(data_folder, day_dir):
    """Assuming a dir corresponding to and named for a day day_dir, (e.g. "day=01") within the data_folder with that day's mobile data files."""
    day = day_dir.split(data_folder)[1]
    filepaths = glob.glob((day_dir + '*[!*.gz]')) # select all the non-zipped mobile data files
    return filepaths, day

def load_data(filepaths, initial_cols, sel_cols, final_cols): 
    """Load in the mobile data and specify the columns"""
    ddf = dd.read_csv(filepaths, names=initial_cols)
    ddf = ddf[sel_cols]
    ddf.columns = final_cols
    return ddf 

def convert_datetime(ddf: dd.DataFrame): #needs work
    """Process timestamp to datetime for dataframe with a "datatime" column with timestamp values. """
    ddf["datetime"] = dd.to_datetime(ddf["datetime"], unit='ms', errors='coerce')
    ddf["datetime"] = ddf["datetime"].dt.tz_localize('UTC').dt.tz_convert('America/Bogota')
    return ddf

def preprocess_mobile(ddf: dd.DataFrame, final_cols: list, minlon , maxlon, minlat, maxlat): #needs work
    """Select only those points within an area of interest and process timestamp to datetime 
    for dataframe with a "datatime" column with timestamp values."""
    ddf = find_within_box(ddf, minlon, maxlon, minlat, maxlat)
    ddf = convert_datetime(ddf)[final_cols]
    df = ddf.compute()
    return df

def find_within_box(ddf, minlon, maxlon, minlat, maxlat):
    """Quick way to filter out points not in a particular rectangular region."""
    box=[minlon,minlat,maxlon,maxlat]
    filtered_ddf = mobilkit.loader.crop_spatial(ddf, box).reset_index()
    return filtered_ddf

#### FUNCTIONS FOR DATABASE ####

def link_database(dir: str, db_name: str):
    db_path = f'{dir}{db_name}.db'
    conn = sqlite3.connect(db_path)
    return db_path, conn

def add_data_to_db(df, conn, table_name, if_exists='replace'): 
    """Add data to database. The options for 'if_exists' include appending ('append') the data or replacing it ('replace')"""
    df.to_sql(table_name, conn, if_exists=if_exists, index=False)
    return

### Database

In [ ]:
import sqlite3

# specify database location and name and create database
input_dir = f'{data_dir}data/'
db_name = 'bogota_mobile_raw'
db_path, conn = link_database(dir=input_dir, db_name=db_name)

In [ ]:
table_name = 'in_Bogota_area'

for i in range(0, len(data_folders)):
    data_folder = data_folders[i]
    day_dirs = glob.glob((data_folder + '*/'))
    for j in trange(0,len(day_dirs)): 
        day_dir = day_dirs[j]
        filepaths, day = get_files(data_folder, day_dir)
        ddf = load_data(filepaths, initial_cols, sel_cols, final_cols)
        df = preprocess_mobile(ddf, final_cols, minlon, maxlon, minlat, maxlat)
        add_data_to_db(df, conn, table_name, if_exists='append')
df.head()

In [ ]:
# Function to execute a query and return the result as a DataFrame
def query_database_ids(query):
    return pd.read_sql_query(query, conn)

# Example query: Retrieve all unique values in the 'device_id' field
query = f"SELECT DISTINCT uid FROM {table_name}"
result = query_database_ids(query)

In [ ]:
# Get the unique values as a list
unique_device_ids = result['uid'].tolist()
unique_device_ids_test = unique_device_ids[:10]

In [ ]:
# Function to perform a query on the database and retrieve the result as a DataFrame
def query_database(query, conn):
    return pd.read_sql_query(query, conn, parse_dates='datetime')


# Example query: Retrieve all rows for a specific device_id
example_id = '84c28cbf-19bd-4a46-b528-197516925af7'
device_id = example_id
query = f"SELECT * FROM {table_name} WHERE uid = '{device_id}'"
result = query_database(query, conn)

In [ ]:
result_dd = dd.from_pandas(result, npartitions=2)
users_stats_df = mobilkit.stats.userStats(result_dd).compute()
users_stats_df


In [ ]:
new_cols = ['total_user_pings', 'daysActive']

# Check if 'new_column' already exists in the table
def check_columns_db(conn, table_name: str, col: str):
    existing_columns = query_database(f"PRAGMA table_info({table_name})", conn)
    if col not in existing_columns['name'].values:
        cursor = conn.cursor()
        cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN {col} REAL")
        cursor.close()
    return 

for i in trange(0, len(new_cols)): 
    check_columns_db(conn, table_name, col=new_cols[i])

# Add the new columns back to the database
result['total_user_pings'], result['daysActive'] = users_stats_df['pings'], users_stats_df['daysActive']
result.to_sql(table_name, conn, if_exists='append', index=False)
result


In [ ]:
def get_user_stats(conn, table_name, uid, npartiions=2):
    query = f"SELECT * FROM {table_name} WHERE uid = '{uid}'"
    result = query_database(query, conn)
    result_dd = dd.from_pandas(result, npartitions=npartiions)
    users_stats_df = mobilkit.stats.userStats(result_dd).compute()
    result['total_user_pings'], result['daysActive'] = users_stats_df['pings'], users_stats_df['daysActive']
    result.to_sql(table_name, conn, if_exists='append', index=False)
    return result

for i in trange(0, len(unique_device_ids_test)): 
    uid = unique_device_ids_test[i]
    users_stats_to_add = get_user_stats(conn, table_name, uid)


In [ ]:
uid = unique_device_ids_test[5]
test = get_user_stats(conn, table_name, uid)
test

In [ ]:
# Function to execute a query and return the result as a DataFrame
def query_database_ids(query):
    return pd.read_sql_query(query, conn)

# Example query: Retrieve all unique values in the 'device_id' field
query = f"SELECT DISTINCT total_user_pings FROM {table_name}"
test2 = query_database_ids(query)
test2

#### Note from July 2023: 
Overall, this accessing and writing to the database using this method doesn't seems as efficient as the parquet method (although maybe both could be combined). I'll focus on the parquet method for the time being (in the tabular_to_parquet.ipynb notebook)